In [1]:
pip install pyaudio

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install SpeechRecognition pydub

In [2]:
import speech_recognition as sr


In [3]:


def audio_start():
    
    r = sr.Recognizer()

    with sr.Microphone() as source:
        # read the audio data from the default microphone 
        r.adjust_for_ambient_noise(source)

        audio_data = r.record(source, duration=5)
        print("Recognizing...")
        
        try:
        # convert speech to text
            text = r.recognize_google(audio_data)
            print("You: " + text)
        except Exception:
            print("No Speech or No Internet")
            audio_start()
            
        if "are you ready to learn" in text:
            
            print("Isaac: Yes! I am listening.")
        
            r.pause_threshold = 5
            audio = r.record(source, duration = 30)
            print("Recognizing...")

            
            try:
                audio_data = r.recognize_google(audio)
                
            except Exception:
                print("No Speech or No Internet")
                
            
    return audio_data       


In [21]:
pip install tflearn

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tflearn in c:\users\gupta\appdata\local\programs\python\python39\lib\site-packages (0.5.0)



In [4]:
import pandas as pd
import numpy as np

import tensorflow as tf
import json
import pickle
import random
import tflearn
import string


import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer =  LancasterStemmer()
from tensorflow.python.framework import ops

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [6]:
nltk.download('punkt')
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
with open ("isaac_dataset.json") as file:
  data=json.load(file)
try:
  with open("data.pickle","rb") as f:
    words,labels,training,output=pickle.load(f)
except:
  words=[]
  labels=[]
  docs_x=[]
  docs_y=[]

  for intent in data["intents"]:
    for pattern in intent["patterns"]:
      wrds=nltk.word_tokenize(pattern)
      words.extend(wrds)
      docs_x.append(wrds)
      docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
      labels.append(intent["tag"])

  words=[stemmer.stem(w.lower()) for w in words if w not in "?"]
  words=sorted(list(set(words)))
  labels=sorted(labels)
  training=[]
  output=[]
  out_empty=[0 for _ in range(len(labels))]

  for x,doc in enumerate(docs_x):
    bag=[]

    wrds=[stemmer.stem(w.lower()) for w in doc]
    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)

    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1

    training.append(bag)
    output.append(output_row)

  training=np.array(training)
  output=np.array(output)

  with open("data.picle","wb") as f:
    pickle.dump((words,labels,training,output),f)

ops.reset_default_graph()

net=tflearn.input_data(shape=[None,len(training[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,len(output[0]),activation='softmax')
net=tflearn.regression(net)

model=tflearn.DNN(net)
try:
  model.load("model.tflearn")
except:
  model=tflearn.DNN(net)
  model.fit(training,output,n_epoch=500,batch_size=8,show_metric=True)
  model.save("model.tflearn")

    
  

Training Step: 6999  | time: 0.042s
| Adam | epoch: 500 | loss: 0.00000 - acc: 0.9987 -- iter: 104/108
Training Step: 7000  | time: 0.046s
| Adam | epoch: 500 | loss: 0.00000 - acc: 0.9988 -- iter: 108/108
--
INFO:tensorflow:c:\Users\gupta\basic\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:c:\Users\gupta\basic\Chatbot\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:c:\Users\gupta\basic\Chatbot\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:c:\Users\gupta\basic\Chatbot\model.tflearn.meta
INFO:tensorflow:200


In [8]:
def bag_of_words(s,words):
  bag=[0 for _ in range(len(words))]
  s_words=nltk.word_tokenize(s)
  s_words=[stemmer.stem(word.lower())for word in s_words]
  for se in s_words:
    for i,w in enumerate(words):
      if w==se:
        bag[i] = 1
  return np.array(bag)

In [9]:
def chat():
  
  print("Isaac: Heyy")
  inp = audio_start()
  print("You: " + inp)

  results=model.predict([bag_of_words(inp,words)])
  results_index=np.argmax(results)
  tag=labels[results_index]

  for tg in data["intents"]:
    if tg["tag"]==tag:
      response=tg['responses']
      print(tag)

  corpus_text = listToString(response)
  print("Issac: "+ (answer(corpus_text,inp)))
  
  return tag

In [10]:
def listToString(response):
   
    str1 = " "
    return (str1.join(response))

In [11]:
def answer(corpus_text,inp):
  corpus_sentences=nltk.sent_tokenize(corpus_text)
  corpus_words=nltk.word_tokenize(corpus_text)

  wn_lemmatizer = nltk.stem.WordNetLemmatizer()

  def lemmatize_data(tokens):
    return [wn_lemmatizer.lemmatize(token) for token in tokens]

  punct_remover=dict((ord(punctuation),None) for punctuation in string.punctuation)

  def get_processed_data(data):
    return lemmatize_data(nltk.word_tokenize(data.lower().translate(punct_remover)))

  corpus_sentences.append(inp)

  word_vectorizer=TfidfVectorizer(tokenizer=get_processed_data)
  corpus_word_vectors=word_vectorizer.fit_transform(corpus_sentences)
  cos_sin_vectors=cosine_similarity(corpus_word_vectors[-1],corpus_word_vectors)
  similar_response_idx=cos_sin_vectors.argsort()[0][-2]

  return corpus_sentences[similar_response_idx]

In [12]:
def ask_ques():
    tag = chat()
    r2 = sr.Recognizer()
    text2 = ""

    with sr.Microphone() as source:
        # read the audio data from the default microphone
        r2.adjust_for_ambient_noise(source)

        audio_data2 = r2.record(source, duration = 20)
        print("Recognizing...")
                    
        try:
            # convert speech to text
            text2 += r2.recognize_google(audio_data2)
            print("You: " + text2)
        except Exception:
            print("No Speech or No Internet")
            

    for tg in data["intents"]:
        if tg["tag"] == tag:
            if text2 in tg["patterns"]:
                print("Yess, Good Job !")
            else:
                print("No, that's the wrong answer")


In [13]:
ask_ques()

Isaac: Heyy
Recognizing...
You: hey Isaac are you ready to learn
Isaac: Yes! I am listening.


KeyboardInterrupt: 